### Initial Set-Up

In [16]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load 
schoolscsv = "Resources/schools_complete.csv"
studentscsv = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(schoolscsv)
student_data = pd.read_csv(studentscsv)

# Combine the data into a single dataset  
combined_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# Preview the combined dataset
combined_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


### District Summary - Final

In [17]:
# Create an overview table that summarizes the district's key metrics.

# Calculate the total number of schools
Tschools = len(combined_data["School ID"].unique())

# Calculate the total number of students
Tstudents = len(combined_data["Student ID"].unique())

# Calculate the total budget
#---need to use the school_data dataset to get correct number (using combined_data will each school's budget multiple times)
Tbudget = school_data["budget"].sum()
    
# Calculate the average math score 
avgMath = combined_data["math_score"].mean()

# Calculate the average reading score
avgReading = combined_data["reading_score"].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
countpassMath = combined_data.loc[combined_data["math_score"]>=70, "math_score"].count()
percentpassMath = (countpassMath/Tstudents)*100

# Calculate the percentage of students with a passing reading score (70 or greater)
countpassReading = combined_data.loc[combined_data["reading_score"]>=70, "reading_score"].count()
percentpassReading = (countpassReading/Tstudents)*100

# Calculate the percentage of students who passed math **and** reading (% Overall Passing)
countpassOverall = combined_data[(combined_data['math_score'] >= 70) & (combined_data['reading_score'] >= 70)]['Student ID'].count()
percentpassOverall = (countpassOverall/Tstudents)*100

# Create a dataframe to hold the above results
districtSum_df = pd.DataFrame ({"Total Schools": [Tschools], 
                                "Total Students": [Tstudents], 
                                "Total Budget": [Tbudget], 
                                "Average Math Score": [avgMath], 
                                "Average Reading Score": [avgReading], 
                                "% Passing Math": [percentpassMath], 
                                "% Passing Reading": [percentpassReading],
                                "% Overall Passing": [percentpassOverall]})

# Give the data cleaner formatting
districtSum_df["Total Students"] = districtSum_df["Total Students"].map("{:,}".format)
districtSum_df["Total Budget"] = districtSum_df["Total Budget"].map("${:,.2f}".format)
districtSum_df["Average Math Score"] = districtSum_df["Average Math Score"].map("{:.2f}".format)
districtSum_df["Average Reading Score"] = districtSum_df["Average Reading Score"].map("{:.2f}".format)
districtSum_df["% Passing Math"] = districtSum_df["% Passing Math"].map("{:.2f}%".format)
districtSum_df["% Passing Reading"] = districtSum_df["% Passing Reading"].map("{:.2f}%".format)
districtSum_df["% Overall Passing"] = districtSum_df["% Overall Passing"].map("{:.2f}%".format)

# Display District Summary Table
districtSum_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


### School Summary - Final

In [18]:
# Create an overview table that summarizes key metrics about each school.

# Rename "school_name" column for cleaner formatting in summary table
school_data = school_data.rename(columns={"school_name": "School Name"})
combined_data = combined_data.rename(columns={"school_name": "School Name"})

# Pull School Type for each school
schTypes = school_data.set_index("School Name")["type"]

# Calculate Total Students for each school
Tstudents_bySch = school_data.set_index("School Name")["size"]

# Calculate Total School Budget for each school
Tbudget_bySch = school_data.set_index("School Name")["budget"]

# Calculate Per Student Budget for each school
PSbudget_bySch = Tbudget_bySch / Tstudents_bySch

### For Avg Math, Reading, Overall Scores###
# Group data by School Name
combined_bySch = combined_data.set_index("School Name").groupby(["School Name"])

# Calculate Average Math Score for each school
avgMath_bySch = combined_bySch["math_score"].mean()

# Calculate Average Reading Score for each school
avgReading_bySch = combined_bySch["reading_score"].mean()

# Calculate % Passing Math for each school
countpassMath_bySch = combined_data[combined_data["math_score"]>=70].groupby(["School Name"])["Student ID"].count()
percentpassMath_bySch = (countpassMath_bySch/Tstudents_bySch)*100

# Calculate % Passing Reading for each school
countpassReading_bySch = combined_data[combined_data["reading_score"]>=70].groupby(["School Name"])["Student ID"].count()
percentpassReading_bySch = (countpassReading_bySch/Tstudents_bySch)*100

# Calculate % Overall Passing (The percentage of students that passed math **and** reading.) for each school
countpassOverall_bySch = combined_data[(combined_data["math_score"] >= 70) & (combined_data["reading_score"] >= 70)].groupby(["School Name"])['Student ID'].count()
percentpassOverall_bySch = (countpassOverall_bySch/Tstudents_bySch)*100


# Create a dataframe to hold the above results
schoolSum_df = pd.DataFrame({"School Type": schTypes,
                             "Total Students": Tstudents_bySch,
                             "Total School Budget": Tbudget_bySch,
                             "Per Student Budget": PSbudget_bySch,
                             "Average Math Score": avgMath_bySch,
                             "Average Reading Score": avgReading_bySch,
                             "% Passing Math": percentpassMath_bySch,
                             "% Passing Math": percentpassMath_bySch,
                             "% Passing Reading": percentpassReading_bySch,
                             "% Overall Passing": percentpassOverall_bySch})
        
# Give the data cleaner formatting
#---need to put the formatted data into a new dataframe in order to use unformatted data for "Scores by" tables later
schoolSum_df_formatted = pd.DataFrame({"School Type": schTypes,
                                       "Total Students": Tstudents_bySch,
                                       "Total School Budget": Tbudget_bySch,
                                       "Per Student Budget": PSbudget_bySch,
                                       "Average Math Score": avgMath_bySch,
                                       "Average Reading Score": avgReading_bySch,
                                       "% Passing Math": percentpassMath_bySch,
                                       "% Passing Math": percentpassMath_bySch,
                                       "% Passing Reading": percentpassReading_bySch,
                                       "% Overall Passing": percentpassOverall_bySch})

schoolSum_df_formatted["Total Students"] = schoolSum_df["Total Students"].map("{:,}".format)
schoolSum_df_formatted["Total School Budget"] = schoolSum_df["Total School Budget"].map("${:,.2f}".format)
schoolSum_df_formatted["Per Student Budget"] = schoolSum_df["Per Student Budget"].map("${:,.2f}".format)
schoolSum_df_formatted["Average Math Score"] = schoolSum_df["Average Math Score"].map("{:.2f}".format)
schoolSum_df_formatted["Average Reading Score"] = schoolSum_df["Average Reading Score"].map("{:.2f}".format)
schoolSum_df_formatted["% Passing Math"] = schoolSum_df["% Passing Math"].map("{:.2f}%".format)
schoolSum_df_formatted["% Passing Reading"] = schoolSum_df["% Passing Reading"].map("{:.2f}%".format)
schoolSum_df_formatted["% Overall Passing"] = schoolSum_df["% Overall Passing"].map("{:.2f}%".format)

# Display School Summary Table
schoolSum_df_formatted

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


### Top Performing Schools (By % Overall Passing) - Final

In [19]:
# Create a table that highlights the top 5 performing schools based on % Overall Passing.

# Find the top five performing schools by sorting the School Summary Table (formatted) by % overall passing.
#---need to have ascending=False to have highest performing listed at the top
top5_df = schoolSum_df_formatted.sort_values("% Overall Passing", ascending=False)

# Display Top 5 Performing Schools Summary Table
top5_df.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


### Bottom Performing Schools (By % Overall Passing) - Final

In [20]:
# Create a table that highlights the bottom 5 performing schools based on % Overall Passing.

# Find the bottom five performing schools by sorting the School Summary Table (formatted) by % overall passing.
bottom5_df = schoolSum_df_formatted.sort_values("% Overall Passing")

# Display Bottom 5 Performing Schools Summary Table
bottom5_df.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


### Math Scores by Grade - Final

In [21]:
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

# Create a pandas series for each grade --- use a conditional statement to extract students from each grade in 'grade' column
grd9 = combined_data.loc[combined_data["grade"] == "9th"]
grd10 = combined_data.loc[combined_data["grade"] == "10th"]
grd11 = combined_data.loc[combined_data["grade"] == "11th"]
grd12 = combined_data.loc[combined_data["grade"] == "12th"]

# Group each series by school and calculate the average Math Score by school
grd9_Math = grd9.groupby(["School Name"])["math_score"].mean()
grd10_Math = grd10.groupby(["School Name"])["math_score"].mean()
grd11_Math = grd11.groupby(["School Name"])["math_score"].mean()
grd12_Math = grd12.groupby(["School Name"])["math_score"].mean()

# Combine the series into a dataframe
mathSum_df = pd.DataFrame({"9th Grade": grd9_Math,
                           "10th Grade": grd10_Math,
                           "11th Grade": grd11_Math,
                           "12th Grade": grd12_Math})

# Give the data cleaner formatting
mathSum_df["9th Grade"] = mathSum_df["9th Grade"].map("{:.2f}".format)
mathSum_df["10th Grade"] = mathSum_df["10th Grade"].map("{:.2f}".format)
mathSum_df["11th Grade"] = mathSum_df["11th Grade"].map("{:.2f}".format)
mathSum_df["12th Grade"] = mathSum_df["12th Grade"].map("{:.2f}".format)

# Take out unnecessary columns from the combined_data dataframe
mathSum_df = mathSum_df[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]]

# Display Math Scores by Grade Summary Table
mathSum_df


,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


### Reading Score by Grade - Final

In [22]:
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

# Group each series made for the Math Summary table by school and calculate the average Reading Score by school
grd9_Reading = grd9.groupby(["School Name"])["reading_score"].mean()
grd10_Reading = grd10.groupby(["School Name"])["reading_score"].mean()
grd11_Reading = grd11.groupby(["School Name"])["reading_score"].mean()
grd12_Reading = grd12.groupby(["School Name"])["reading_score"].mean()

# Combine the series into a dataframe
readingSum_df = pd.DataFrame({"9th Grade": grd9_Reading,
                              "10th Grade": grd10_Reading,
                              "11th Grade": grd11_Reading,
                              "12th Grade": grd12_Reading})

# Give the data cleaner formatting
readingSum_df["9th Grade"] = readingSum_df["9th Grade"].map("{:.2f}".format)
readingSum_df["10th Grade"] = readingSum_df["10th Grade"].map("{:.2f}".format)
readingSum_df["11th Grade"] = readingSum_df["11th Grade"].map("{:.2f}".format)
readingSum_df["12th Grade"] = readingSum_df["12th Grade"].map("{:.2f}".format)

# Again, take out unnecessary columns from the combined_data dataframe
readingSum_df = readingSum_df[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]]

# Display Reading Scores by Grade Summary Table
readingSum_df


,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


### Scores by School Spending - Final

In [23]:
# Create a table that breaks down school performances based on average Spending Ranges (Per Student). 

# Use 4 reasonable bins to group school spending
bins_Spnd = [0, 599.00, 629.99, 649.99, 999.99]
labels_Spnd = ["<$600", "$600-629", "$630-649", "$650+"]

# Take relevant columns from School Summary table and put them in a new dataframe to use for this table
schoolSum_Spnd = schoolSum_df.loc[:,["Average Math Score",
                                     "Average Reading Score",
                                     "% Passing Math",
                                     "% Passing Reading",
                                     "% Overall Passing"]]

# Cut the data from schoolSum_df "Per Student Budget" column into the bins
# Place the groups into a new column in the schoolSum_Spnd dataframe
schoolSum_Spnd["Spending Ranges (Per Student)"] = pd.cut(schoolSum_df["Per Student Budget"], bins_Spnd, labels=labels_Spnd)

# Group schoolSum_Spnd by Spending Ranges (Per Student), and calculate the means for each to get Scores by Spending Ranges
scores_bySpnd_df = schoolSum_Spnd.groupby("Spending Ranges (Per Student)").mean()

# Give the data cleaner formatting
scores_bySpnd_df["Average Math Score"] = scores_bySpnd_df["Average Math Score"].map("{:.2f}".format)
scores_bySpnd_df["Average Reading Score"] = scores_bySpnd_df["Average Reading Score"].map("{:.2f}".format)
scores_bySpnd_df["% Passing Math"] = scores_bySpnd_df["% Passing Math"].map("{:.2f}%".format)
scores_bySpnd_df["% Passing Reading"] = scores_bySpnd_df["% Passing Reading"].map("{:.2f}%".format)
scores_bySpnd_df["% Overall Passing"] = scores_bySpnd_df["% Overall Passing"].map("{:.2f}%".format)

# Display schoolSum_bySpnd
scores_bySpnd_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$600,83.46,83.93,93.46%,96.61%,90.37%
$600-629,81.90,83.16,87.13%,92.72%,81.42%
$630-649,78.52,81.62,73.48%,84.39%,62.86%
$650+,77.00,81.03,66.16%,81.13%,53.53%


### Scores by School Size - Final 

In [24]:
# Create a table that breaks down school performances based on school size. 

# Use 3 reasonable bins to group school size
bins_Size = [0, 999, 1999, 5000]
labels_Size = ["Small (<1000)", "Medium (1000-1999)", "Large (2000-5000)"]

# Take relevant columns from School Summary table and put them in a new dataframe to use for this table
schoolSum_Size = schoolSum_df.loc[:,["Average Math Score",
                                     "Average Reading Score",
                                     "% Passing Math",
                                     "% Passing Reading",
                                     "% Overall Passing"]]

# Cut the data from schoolSum_Size "Total Students" column into the bins
# Place the groups into a new column in the schoolSum_Size dataframe
schoolSum_Size["School Size"] = pd.cut(schoolSum_df["Total Students"], bins_Size, labels=labels_Size)

# Group schoolSum_Size by School Size, and calculate the means for each to get Scores by School Size
scores_bySize_df = schoolSum_Size.groupby("School Size").mean()

# Give the data cleaner formatting
scores_bySize_df["Average Math Score"] = scores_bySize_df["Average Math Score"].map("{:.2f}".format)
scores_bySize_df["Average Reading Score"] = scores_bySize_df["Average Reading Score"].map("{:.2f}".format)
scores_bySize_df["% Passing Math"] = scores_bySize_df["% Passing Math"].map("{:.2f}%".format)
scores_bySize_df["% Passing Reading"] = scores_bySize_df["% Passing Reading"].map("{:.2f}%".format)
scores_bySize_df["% Overall Passing"] = scores_bySize_df["% Overall Passing"].map("{:.2f}%".format)

# Display schoolSum_bySpnd
scores_bySize_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-1999),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


### Scores by School Type - Final

In [25]:
# Create a table that breaks down school performances based on school type.

# Take relevant columns from School Summary table and put them in a new dataframe to use for this table
schoolSum_Type = schoolSum_df.loc[:,["School Type",
                                     "Average Math Score",
                                     "Average Reading Score",
                                     "% Passing Math",
                                     "% Passing Reading",
                                     "% Overall Passing"]]

# Group schoolSum_Type by School Type, and calculate the means for each to get Scores by School Type 
scores_byType_df = schoolSum_Type.groupby("School Type").mean()

# Give the data cleaner formatting
scores_byType_df["Average Math Score"] = scores_byType_df["Average Math Score"].map("{:.2f}".format)
scores_byType_df["Average Reading Score"] = scores_byType_df["Average Reading Score"].map("{:.2f}".format)
scores_byType_df["% Passing Math"] = scores_byType_df["% Passing Math"].map("{:.2f}%".format)
scores_byType_df["% Passing Reading"] = scores_byType_df["% Passing Reading"].map("{:.2f}%".format)
scores_byType_df["% Overall Passing"] = scores_byType_df["% Overall Passing"].map("{:.2f}%".format)

# Display schoolSum_bySpnd
scores_byType_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
